In [2]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('length of the dataset in characters:', len(text))

length of the dataset in characters: 1115394


In [3]:
# Let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size) 


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] # encoder: takes a string, outputs a list of integers
decode = lambda l: ''.join(itos[ix] for ix in l) # decoder: takes a list of integers, outputs a string

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


what is the difference between the two below code?

```decode = lambda l: ''.join([itos[ix] for ix in l])```

```decode = lambda l: ''.join(itos[ix] for ix in l)```


Let me explain the difference between these two code snippets. 

While they achieve the same result, there's a subtle syntactic difference:

decode = lambda l: ''.join([itos[ix] for ix in l])

This version uses a list comprehension [itos[ix] for ix in l]
It creates a full list in memory first, then joins it
The square brackets [] indicate that a new list object is being created

decode = lambda l: ''.join(itos[ix] for ix in l)

This version uses a generator expression (itos[ix] for ix in l)
It generates values one at a time without creating a full list
No square brackets means it's a generator expression
The main practical differences are:
Memory efficiency: The generator version (second one) is more memory efficient because it doesn't create an intermediate list. It generates each value on-the-fly as needed by join().

Performance:
For small sequences, the list comprehension might be slightly faster since it has less overhead

For large sequences, the generator version is better as it uses less memory
In most cases, the difference in performance would be negligible, but if you're working with very large sequences, the generator version (second one) would be the better choice due to its memory efficiency. Both will produce exactly the same output - they just differ in how they get there.

In [6]:
# tokenize the enitre dataset and store it in a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
# data = data.unsqueeze(0)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# split the dataset into train and validation sets (90%)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [8]:
# Time Dimention of the input tensors feeding into the transformer

block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target: {target}")

when input is tensor([18]), the target: 47
when input is tensor([18, 47]), the target: 56
when input is tensor([18, 47, 56]), the target: 57
when input is tensor([18, 47, 56, 57]), the target: 58
when input is tensor([18, 47, 56, 57, 58]), the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58


In [9]:
# Batch Dimention
# We are going to have mini batches of multiple chunks of text all stacked up in a single tensor
# We want to process multiple chunks at the same time since we are using GPUs which are good at parallel processing

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and taregs y
    data = train_data if split == 'train' else val_data
    # The reason we subtract block_size from len(data) in torch.randint(len(data) - block_size, (batch_size,)) is to ensure we can always get a complete sequence of length block_size starting from any randomly chosen index.
    ix = torch.randint(len(data)- block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('-----')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t+1]
        target = yb[b, t]
        print("When input is ", context.tolist(), "the target: ", target.tolist())

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
When input is  [24] the target:  43
When input is  [24, 43] the target:  58
When input is  [24, 43, 58] the target:  5
When input is  [24, 43, 58, 5] the target:  57
When input is  [24, 43, 58, 5, 57] the target:  1
When input is  [24, 43, 58, 5, 57, 1] the target:  46
When input is  [24, 43, 58, 5, 57, 1, 46] the target:  43
When input is  [24, 43, 58, 5, 57, 1, 46, 43] the target:  39
When input is  [44] the target:  53
When input is  [44, 53] the target:  56
When input is  [44, 53, 56] the target:  1
When input is  [44, 53, 56, 1] the target:  58
When input is  [44, 53, 56, 1, 58] the targ

In [10]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)
        
        if targets is None:
            loss = None
        else:
            # pytorch cross entropy expects inputs of shape (B, C, T) so we need to reshape logits
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # this is equivalent to targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions 
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) # we must pluck out the last element in the time dimention
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
print(yb.shape)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item()) # the expected loss is -ln(1/65)
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
# the generated text is garbage since it is in totally random level

torch.Size([4, 8])
torch.Size([32, 65])
4.878634929656982

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())



2.382369041442871


In [13]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))
# this is a fairly simple model since the tokens are not talking to each other.
# Given any context we only llok at the very laast character to make the prediction for the next character.
# So, the tokens must start talking to each other, and they must know what is included in the context so that they can make better predictions about what will come next.




lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


#### The mathematical trick in self attention

In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channel
x = torch.randn(B, T, C)
print(x.shape)
x.mean(dim=-1)

# What we try to achieve is that make the tokens at each row of the batch to talk to each other.
# But not in a way that for example the token at the fifth location talks to 6th, 7th, ... locations since those are future tokens in the sequence
# It should only talk to the 4th, 3rd, 2nd and 1st tokens
# So, the information only flows from previous context to the current time step, we can't get any information from future because we try to predict the future
# One way for the tokens to talk to each other is to take the avergae of the previous token as the feature vector for the t-th token

# We want x[b, t] = mean_{i<=t} x[b, i]
# version 1
xbow = torch.zeros((B, T, C)) # bow stands for Bag of Words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t, C)
        xbow[b, t] = torch.mean(xprev, 0)

torch.Size([4, 8, 2])


In [20]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [21]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [ ]:
# This current version is really inefficient since we are using two nested for loops
# We can use matrix multiplication 

# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ----> (B, T, T) @ (B, T, C) this is a batch multiplication the result is (B, T, C)
torch.allclose(xbow, xbow2)

True

In [ ]:
# version 3 
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf')) # the future can not communicate with past
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

True

You can do weighted aggregation of the past elements by using matrix multiplication. The elements in the tril tells you how much of each element gives us into this position